# Laboratorio 1 Phishing detection
## Security Data Science
### Universidad del valle de Guatemala

Abner Ivan Garcia - 21285
Oscar Esteba Donis - 21610


In [6]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

# Parte 1 - Ingeniería de características

In [7]:
df = pd.read_csv('dataset_pishing.csv')
print (df.shape)
print(df.head())
print(df.info())

# ejemplos de dominios del dataset (legitimos)
df[df.status == 'legitimate'].sample(5)

(11430, 2)
                                                 url      status
0              http://www.crestonwood.com/router.php  legitimate
1  http://shadetreetechnology.com/V4/validation/a...    phishing
2  https://support-appleld.com.secureupdate.duila...    phishing
3                                 http://rgipt.ac.in  legitimate
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     11430 non-null  object
 1   status  11430 non-null  object
dtypes: object(2)
memory usage: 178.7+ KB
None


,url,status
3692,http://www.naturesbasket.co.in/,legitimate
2989,http://www.binbert.com/blog/2010/05/difference...,legitimate
8403,http://www.optus.com.au/shop/support/answer?an...,legitimate
7680,https://wiki.ezvid.com/best-wireless-routers,legitimate
1761,http://www.themitchelli.com/,legitimate


In [8]:
# Ejemplos de dominios del dataset (phishing)
df[df.status == 'phishing'].sample(5)

,url,status
1121,http://webfiddle.net/cats-d8c4vu/www.ebay.com/...,phishing
1942,http://www.alqurancampus.com/wp-admin/network/...,phishing
7228,http://andreacostafisio.com.br/wp-content/plug...,phishing
11126,http://abcd.idknet.com/,phishing
3592,http://inbioma.pe/Rechnung-08232502429997968&a...,phishing


In [9]:
# Contar la cantidad de observaciones para cada etiqueta de la columna 'status'
conteo_status = df.status.value_counts()

# Mostrando los resultados
print("La cantidad de observaciones para cada etiqueta de la columna 'status' de legitimo es:", conteo_status['legitimate'])
print("La cantidad de observaciones para cada etiqueta de la columna 'status' de phishing es:", conteo_status['phishing']) 

La cantidad de observaciones para cada etiqueta de la columna 'status' de legitimo es: 5715
La cantidad de observaciones para cada etiqueta de la columna 'status' de phishing es: 5715


R// Si esta balanceado

## Derivación de caracteristicas

#### 1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?
R// El análisis de una URL destaca por su eficiencia, especificidad y adaptabilidad. A diferencia de analizar el tiempo de vida de un dominio o las características generales de una página web, la URL proporciona acceso directo y conciso a la información relevante. Permite un análisis más preciso y específico, enfocándose en elementos particulares como palabras clave o estructura, sin necesidad de procesar datos adicionales. Además, su flexibilidad permite adaptarse a diversos contextos y objetivos, desde clasificar páginas web hasta detectar malware, superando las limitaciones de otros análisis de datos menos versátiles.

#### 2. ¿Qué características de una URL son más prometedoras para la detección de phishing?
R//
1. Longitud de la URL
2. Cantidad de digitos en la URL
3. Entropia (Shannon) de la URL
4. Entropia (Relativa) de la URL
5. Proporcion entre vocales y consonantes
6. Posicion del primer digito en la URL
7. Presencia de direccion IP en la URL
8. Caracteres especiales en la URL
9. Presencia de puertos en la URL
10. Cantidad de subdominios
11. Cantidad de terminos comunes en URL (www, .com, http, https, etc)
12. Presencia de guiones bajos en la URL
13. Presencia de guiones en la URL
14. URL acortada
15. Subdominios abnormales

En base a la respuesta anterior escriba al menos quince funciones basadas en los artículos, para
derivar características que un modelo pueda utilizar y añada dichas características al dataset original. Incluya dentro de las quince funciones, la entropía de Shanon y relativa.

In [18]:
import re
import math
import tldextract
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
import numpy as np

def url_length(url):
    return len(url)

def digit_count(url):
    return sum(c.isdigit() for c in url)

def shannon_entropy(url):
    """
    Calculate Shannon entropy of URL string
    Returns entropy value between 0 and log2(len(unique_chars))
    """
    try:
        # Calculate probability of each character
        prob = [float(url.count(c)) / len(url) for c in dict.fromkeys(list(url))]
        # Calculate entropy using probability distribution
        entropy = -sum(p * np.log2(p) for p in prob)
        return entropy
    except:
        return 0

def relative_entropy(url):
    """
    Calculate relative entropy (Kullback-Leibler divergence) of URL
    compared to expected character frequencies in legitimate URLs
    """
    try:
        # Preprocess URL
        url = url.replace('www.', '')
        url = url.split('://')[-1]
        url = url.split('.')[0]
        url = url.lower()
        
        # Expected character probabilities (based on Alexa Top 1M domains)
        probabilities = {
            '-': 0.013342298553905901,
            'a': 0.07333590631143488,
            'b': 0.04293204925644953,
            'c': 0.027385633133525503,
            'd': 0.02769469202658208,
            'e': 0.07086192756262588,
            'f': 0.01249653250998034,
            'g': 0.038516276096631406,
            'h': 0.024017645001386995,
            'i': 0.060447396668797414,
            'j': 0.007082725266242929,
            'k': 0.01659570875496002,
            'l': 0.05815885325582237,
            'm': 0.033884915513851865,
            'n': 0.04753175014774523,
            'o': 0.09413783122067709,
            'p': 0.042555148167356144,
            'q': 0.0017231917793349655,
            'r': 0.06460084667060655,
            's': 0.07214640647425614,
            't': 0.06447722311338391,
            'u': 0.034792493336388744,
            'v': 0.011637198026847418,
            'w': 0.013318176884203925,
            'x': 0.003170491961453572,
            'y': 0.016381628936354975,
            'z': 0.004715786426736459
        }
        
        # Calculate observed frequencies
        length = float(len(url))
        if length == 0:
            return 0
            
        char_counts = Counter(url)
        
        # Calculate relative entropy
        entropy = 0.0
        for char, count in char_counts.items():
            if char in probabilities:
                observed = count / length
                expected = probabilities[char]
                entropy += observed * np.log2(observed / expected)
        
        return entropy
    except:
        return 0

def vowel_consonant_ratio(url):
    vowels = sum(1 for c in url if c.lower() in 'aeiou')
    consonants = sum(1 for c in url if c.isalpha() and c.lower() not in 'aeiou')
    return vowels / consonants if consonants > 0 else 0

def first_digit_position(url):
    match = re.search(r'\d', url)
    return match.start() if match else -1

def contains_ip(url):
    return bool(re.search(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', url))

def special_chars_count(url):
    return len(re.findall(r'[^a-zA-Z0-9]', url))

def contains_port(url):
    parsed = urlparse(url)
    return bool(parsed.port)

def subdomain_count(url):
    extracted = tldextract.extract(url)
    return extracted.subdomain.count('.')

def common_terms_count(url):
    common_terms = {'www', 'http', 'https', 'com', 'net', 'org', 'edu'}
    return sum(url.lower().count(term) for term in common_terms)

def contains_underscore(url):
    return '_' in url

def contains_dash(url):
    return '-' in url

def is_shortened(url):
    shortened_services = {"bit.ly", "goo.gl", "tinyurl.com", "t.co", "ow.ly", "is.gd", "buff.ly"}
    extracted = tldextract.extract(url)
    return extracted.domain in shortened_services

def abnormal_subdomains(url):
    extracted = tldextract.extract(url)
    return len(extracted.subdomain) > 20

# Preprocesamiento de datos

In [19]:
df['url_length'] = df['url'].apply(url_length)
df['digit_count'] = df['url'].apply(digit_count)
df['shannon_entropy'] = df['url'].apply(shannon_entropy)
df['relative_entropy'] = df['url'].apply(relative_entropy)
df['vowel_consonant_ratio'] = df['url'].apply(vowel_consonant_ratio)
df['first_digit_position'] = df['url'].apply(first_digit_position)
df['contains_ip'] = df['url'].apply(contains_ip)
df['special_chars_count'] = df['url'].apply(special_chars_count)
df['contains_port'] = df['url'].apply(contains_port)
df['subdomain_count'] = df['url'].apply(subdomain_count)
df['common_terms_count'] = df['url'].apply(common_terms_count)
df['contains_underscore'] = df['url'].apply(contains_underscore)
df['contains_dash'] = df['url'].apply(contains_dash)
df['is_shortened'] = df['url'].apply(is_shortened)
df['abnormal_subdomains'] = df['url'].apply(abnormal_subdomains)

print(df.head())

                                                 url      status  url_length  \
0              http://www.crestonwood.com/router.php  legitimate          37   
1  http://shadetreetechnology.com/V4/validation/a...    phishing          77   
2  https://support-appleld.com.secureupdate.duila...    phishing         126   
3                                 http://rgipt.ac.in  legitimate          18   
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate          55   

   digit_count  shannon_entropy  relative_entropy  vowel_consonant_ratio  \
0            0         3.787043          1.221603               0.363636   
1           17         4.419864          0.792675               0.827586   
2           19         4.753412          1.117812               0.517241   
3            0         3.461320          1.919763               0.300000   
4            0         4.097662          1.778614               0.363636   

   first_digit_position  contains_ip  special_chars_count  con

In [20]:
df_final = df
df_final = df_final.drop(['url'], axis=1)
df_final.to_csv('dataset_pishing_final.csv', index=False)
df_final.head()

,status,url_length,digit_count,shannon_entropy,relative_entropy,vowel_consonant_ratio,first_digit_position,contains_ip,special_chars_count,contains_port,subdomain_count,common_terms_count,contains_underscore,contains_dash,is_shortened,abnormal_subdomains
0,legitimate,37,0,3.787043,1.221603,0.363636,-1,False,7,False,0,3,False,False,False,False
1,phishing,77,17,4.419864,0.792675,0.827586,32,False,7,False,0,2,False,False,False,False
2,phishing,126,19,4.753412,1.117812,0.517241,62,False,19,False,2,4,True,True,False,True
3,legitimate,18,0,3.461320,1.919763,0.300000,-1,False,5,False,0,1,False,False,False,False
4,legitimate,55,0,4.097662,1.778614,0.363636,-1,False,10,False,0,3,False,True,False,False


In [21]:
df_final.dtypes

status                    object
url_length                 int64
digit_count                int64
shannon_entropy          float64
relative_entropy         float64
vowel_consonant_ratio    float64
first_digit_position       int64
contains_ip                 bool
special_chars_count        int64
contains_port               bool
subdomain_count            int64
common_terms_count         int64
contains_underscore         bool
contains_dash               bool
is_shortened                bool
abnormal_subdomains         bool
dtype: object

# Seleccion de Caracteristicas